In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount= True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Imports and data loading

In [2]:
!unzip "/content/drive/My Drive/AI_and_ethics/Gamergate_Dataset.zip"

Archive:  /content/drive/My Drive/AI_and_ethics/Gamergate_Dataset.zip
  inflating: Gamergate_Dataset/Labels_for_7320_users/gamergate_opposer_feminist_usernames  
  inflating: Gamergate_Dataset/Labels_for_7320_users/unaffiliated_usernames  
  inflating: Gamergate_Dataset/Labels_for_7320_users/gamergate_supporter_misogynist_usernames  
  inflating: Gamergate_Dataset/Gamergate.json  


In [0]:
import json
import pprint
import numpy as np
import pandas as pd
import os

In [0]:
tweets =[]
with open('Gamergate_Dataset/Gamergate.json','r') as f:
    for line in f.readlines():
        tweets.append(json.loads(line))

In [0]:
with open('Gamergate_Dataset/Labels_for_7320_users/gamergate_opposer_feminist_usernames', 'r') as f:
    opposer =f.read().splitlines()
with open('Gamergate_Dataset/Labels_for_7320_users/gamergate_supporter_misogynist_usernames','r') as f:
    supporter = f.read().splitlines()
with open('Gamergate_Dataset/Labels_for_7320_users/unaffiliated_usernames','r') as f:
    unaffiliated = f.read().splitlines()

In [0]:
allusers = opposer + unaffiliated + supporter

#Graph creation

In [0]:
import networkx as nx
# import matplotlib.pyplot as plt

In [8]:
nx.__version__

'2.4'

In [0]:
G = nx.Graph()

In [0]:
G.add_nodes_from(allusers)

In [0]:
for tweet in tweets:
    username = tweet['user']['screen_name']
    if username not in allusers:
        continue
    if 'retweeted_status' not in tweet:
        continue
    retweeteduser = tweet['retweeted_status']['user']['screen_name']
    if retweeteduser not in allusers:
        continue
    G.add_edge(username,retweeteduser)

In [0]:
G = G.to_undirected()

#Louvain

In [0]:
import community as community_louvain

In [0]:
partition = community_louvain.best_partition(G, randomize=False)

In [15]:
len(partition)

7320

In [0]:
import numpy as np

In [17]:
tmp = np.unique(list(partition.values()), return_counts=True)
tmp = list(zip(*tmp))
major_communities = sorted(tmp, key=lambda x: x[1], reverse=True)[:4]
# sum([i[1] for i in tmp])/7320*100
major_communities = dict(major_communities)
major_communities

{0: 2723, 1: 323, 2: 1484, 3: 1011}

In [0]:
communities = {k: [] for k, _ in major_communities.items()}
for user, community in partition.items():
    if community in major_communities:
        communities[community].append(user)

In [19]:
communities.keys()

dict_keys([0, 2, 3, 1])

#XGboost

In [0]:
del dftrain
del dftest

In [0]:
dftest = pd.read_pickle("/content/drive/My Drive/AI_and_ethics/test4.pkl")
dftrain = pd.read_pickle("/content/drive/My Drive/AI_and_ethics/train4.pkl")


In [0]:
dftrain = dftrain.drop(['user_id'],axis =1)
dftest= dftest.drop(['user_id'],axis =1)

In [22]:
dftrain

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,49119,49120,49121,49122,49123,49124,49125,49126,49127,49128,49129,49130,49131,49132,49133,49134,49135,49136,49137,49138,49139,49140,49141,49142,49143,49144,49145,49146,49147,49148,49149,49150,49151,49152,49153,49154,49155,49156,49157,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5536,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5537,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5538,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5539,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [23]:
from xgboost import XGBClassifier

xgbc = XGBClassifier()
xgbc.fit(dftrain.iloc[:,:-1], dftrain.iloc[:,-1])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
pred = xgbc.predict(dftest.iloc[:,:-1])

In [27]:
from collections import Counter
dict(Counter(pred))

{0: 1445, 2: 296, 3: 38}

In [0]:
dftest['predlabel'] = pred

In [30]:
dftest.apply(lambda row: communities[row.predlabel].append(row.user_id),axis = 1 )

0       None
1       None
2       None
3       None
4       None
        ... 
1774    None
1775    None
1776    None
1777    None
1778    None
Length: 1779, dtype: object

In [0]:
majorities = {}

In [0]:
for community in communities:
   for community, users in communities.items():
    o = 0
    s = 0
    u = 0
    for sn in users:
        if sn in opposer:
            o += 1
        elif sn in supporter:
            s += 1
        else:
            u += 1
    if max(s, o, u) == s:
        majorities[community] = 1
    elif max(s, o, u) == o:
        majorities[community] = 0
    else:
        majorities[community] = 2 

In [33]:
majorities

{0: 0, 1: 1, 2: 1, 3: 1}

In [0]:
user_pred_labels = {}
for community, users in communities.items():
    for user in users:
        user_pred_labels[user] = majorities[community]

In [0]:
sorted(user_pred_labels)

In [0]:
user_actual_labels = {}
for user in supporter:
    user_actual_labels[user] = 1
for user in opposer:
    user_actual_labels[user] = 0
for user in unaffiliated:
    user_actual_labels[user] = 2

In [37]:
dict(sorted(user_actual_labels.items())).values()

dict_values([0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 2, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 2, 0, 1, 1, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 2, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 2, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 2, 2, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 2, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 2, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 2, 1, 0, 0, 0, 1, 0, 2, 0, 0, 1, 1, 1, 

In [0]:
dfres = pd.DataFrame({'gt':list(dict(sorted(user_actual_labels.items())).values()),'pred': list(dict(sorted(user_pred_labels.items())).values())})

In [0]:
from sklearn.metrics import classification_report, jaccard_score

In [41]:
print(classification_report(dfres['gt'],dfres['pred']))

              precision    recall  f1-score   support

           0       0.67      0.74      0.70      3778
           1       0.65      0.66      0.66      3117
           2       0.00      0.00      0.00       425

    accuracy                           0.66      7320
   macro avg       0.44      0.47      0.45      7320
weighted avg       0.62      0.66      0.64      7320



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
from sklearn.preprocessing import OneHotEncoder

In [0]:
onehot = OneHotEncoder(handle_unknown='ignore')
jaccard_gt = onehot.fit_transform([[i] for i in list(dfres['gt'])]).toarray()
jaccard_pred = onehot.transform([[i] for i in list(dfres['pred'])]).toarray()

In [46]:
print(jaccard_score(jaccard_gt, jaccard_pred, average='samples'))

0.6624316939890711
